In [ ]:
#imports

import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.metrics import precision_score
import nltk
import random
nltk.download('stopwords')

from nltk.corpus import stopwords
import string
stopset = stopwords.words('english') + list(string.punctuation)
nltk.download('punkt')
import re

#rom google.colab import files
#uploaded = files.upload()

import io

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#!pip install transformers sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 85 kB 3.7 MB/s 
     |████████████████████████████████| 182 kB 61.6 MB/s 
     |████████████████████████████████| 7.6 MB 68.6 MB/s 
     |████████████████████████████████| 1.3 MB 80.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f5aef1d62c871bc8ac0035bf565905cdda230c9d41701c1f4892ffff967d2e88
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [ ]:
#more imports

import torch
import transformers as ppb

In [ ]:
#load data

drive.mount('/content/drive', force_remount=True)
nyt = pd.read_csv('/content/drive/MyDrive/NYT.csv')

Mounted at /content/drive


In [ ]:
#split data into training, validation, test sets

years = [int(i[-4:]) for i in nyt['date']]
nyt['year'] = years
nyt_train = nyt[nyt['year'] <= 2020]
nyt_val = nyt[nyt['year'] == 2021]
nyt_test = nyt[nyt['year'] == 2022]

In [ ]:
#train/val answers/clues

train_answers = [k for j in [list(eval(i)) for i in nyt_train['answer']] for k in j]
val_answers = [k for j in [list(eval(i)) for i in nyt_val['answer']] for k in j]
train_clues = [k for j in [list(eval(i)) for i in nyt_train['clue']] for k in j]
val_clues = [k for j in [list(eval(i)) for i in nyt_val['clue']] for k in j]

In [ ]:
#convert to arrays

train_answers = np.array(train_answers)
val_answers = np.array(val_answers)
train_clues = np.array(train_clues)
val_clues = np.array(val_clues)

In [ ]:
#test answers and clues

test_answers = [k for j in [list(eval(i)) for i in nyt_test['answer']] for k in j]
test_clues = [k for j in [list(eval(i)) for i in nyt_test['clue']] for k in j]

test_answers = np.array(test_answers)
test_clues = np.array(test_clues)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

In [ ]:
#load model, configure, layer word embedding, pooling, and dense model

from sentence_transformers import SentenceTransformer, models
from torch import nn

word_embedding_model = models.Transformer('bert-base-uncased', model_args={'hidden_act':'gelu_new', 'hidden_dropout_prob':0.1, \
                                                                           'attention_probs_dropout_prob':0.1}, max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
dense_model = models.Dense(in_features=pooling_model.get_sentence_embedding_dimension(), out_features=256, activation_function=nn.ReLU())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model])

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#pair clues/answers for input into training

from sentence_transformers import InputExample

train_examples = [InputExample(texts=[train_clues[i], train_answers[i]]) for i in range(len(train_clues))]

In [ ]:
#load with batchsize

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=384)

In [ ]:
#add loss function

from sentence_transformers import losses

train_loss = losses.MultipleNegativesRankingLoss(model)

In [ ]:
#train model

from sentence_transformers import SentenceTransformer, models


torch.cuda.empty_cache()
#pool = model.start_multi_process_pool('cuda')
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=5, warmup_steps=100) 

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

Iteration:   0%|          | 0/2100 [00:00<?, ?it/s]

In [ ]:
#save trained model in case of crash

import pickle
filename = '/content/drive/MyDrive/gelunewbertbaseuncased_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
#reload model

import pickle
filename = '/content/drive/MyDrive/gelunewbertbaseuncased_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
#initialize embedding arrays

train_clue_emb = []
train_answer_emb = []
val_clue_emb = []
val_answer_emb = []
test_clue_emb = []
test_answer_emb = []

In [ ]:
#compute test embeddings

for i in range(len(test_clues)):
  #print(i)
  test_clue_emb.append(loaded_model.encode(test_clues[i]))
  test_answer_emb.append(loaded_model.encode(test_answers[i]))

Streaming output truncated to the last 5000 lines.
19736
19737
19738
19739
19740
19741
19742
19743
19744
19745
19746
19747
19748
19749
19750
19751
19752
19753
19754
19755
19756
19757
19758
19759
19760
19761
19762
19763
19764
19765
19766
19767
19768
19769
19770
19771
19772
19773
19774
19775
19776
19777
19778
19779
19780
19781
19782
19783
19784
19785
19786
19787
19788
19789
19790
19791
19792
19793
19794
19795
19796
19797
19798
19799
19800
19801
19802
19803
19804
19805
19806
19807
19808
19809
19810
19811
19812
19813
19814
19815
19816
19817
19818
19819
19820
19821
19822
19823
19824
19825
19826
19827
19828
19829
19830
19831
19832
19833
19834
19835
19836
19837
19838
19839
19840
19841
19842
19843
19844
19845
19846
19847
19848
19849
19850
19851
19852
19853
19854
19855
19856
19857
19858
19859
19860
19861
19862
19863
19864
19865
19866
19867
19868
19869
19870
19871
19872
19873
19874
19875
19876
19877
19878
19879
19880
19881
19882
19883
19884
19885
19886
19887
19888
19889
19890
19891
19892
19893
1

In [ ]:
#compute validation embeddings

for i in range(len(val_clues)):
  #print(i)
  val_clue_emb.append(loaded_model.encode(val_clues[i]))
  val_answer_emb.append(loaded_model.encode(val_answers[i]))

Streaming output truncated to the last 5000 lines.
23721
23722
23723
23724
23725
23726
23727
23728
23729
23730
23731
23732
23733
23734
23735
23736
23737
23738
23739
23740
23741
23742
23743
23744
23745
23746
23747
23748
23749
23750
23751
23752
23753
23754
23755
23756
23757
23758
23759
23760
23761
23762
23763
23764
23765
23766
23767
23768
23769
23770
23771
23772
23773
23774
23775
23776
23777
23778
23779
23780
23781
23782
23783
23784
23785
23786
23787
23788
23789
23790
23791
23792
23793
23794
23795
23796
23797
23798
23799
23800
23801
23802
23803
23804
23805
23806
23807
23808
23809
23810
23811
23812
23813
23814
23815
23816
23817
23818
23819
23820
23821
23822
23823
23824
23825
23826
23827
23828
23829
23830
23831
23832
23833
23834
23835
23836
23837
23838
23839
23840
23841
23842
23843
23844
23845
23846
23847
23848
23849
23850
23851
23852
23853
23854
23855
23856
23857
23858
23859
23860
23861
23862
23863
23864
23865
23866
23867
23868
23869
23870
23871
23872
23873
23874
23875
23876
23877
23878
2

In [ ]:
#compute training embeddings

for i in range(len(train_clues)):
  #print(i)
  train_clue_emb.append(loaded_model.encode(train_clues[i]))
  train_answer_emb.append(loaded_model.encode(train_answers[i]))

Streaming output truncated to the last 5000 lines.
801218
801219
801220
801221
801222
801223
801224
801225
801226
801227
801228
801229
801230
801231
801232
801233
801234
801235
801236
801237
801238
801239
801240
801241
801242
801243
801244
801245
801246
801247
801248
801249
801250
801251
801252
801253
801254
801255
801256
801257
801258
801259
801260
801261
801262
801263
801264
801265
801266
801267
801268
801269
801270
801271
801272
801273
801274
801275
801276
801277
801278
801279
801280
801281
801282
801283
801284
801285
801286
801287
801288
801289
801290
801291
801292
801293
801294
801295
801296
801297
801298
801299
801300
801301
801302
801303
801304
801305
801306
801307
801308
801309
801310
801311
801312
801313
801314
801315
801316
801317
801318
801319
801320
801321
801322
801323
801324
801325
801326
801327
801328
801329
801330
801331
801332
801333
801334
801335
801336
801337
801338
801339
801340
801341
801342
801343
801344
801345
801346
801347
801348
801349
801350
801351
801352
8013

In [ ]:
#export embeddings to csv

pd.DataFrame(train_clue_emb).to_csv('/content/drive/MyDrive/berttrainclue.csv',index=False)
pd.DataFrame(train_answer_emb).to_csv('/content/drive/MyDrive/berttrainanswer.csv',index=False)
pd.DataFrame(val_clue_emb).to_csv('/content/drive/MyDrive/bertvalclue.csv',index=False)
pd.DataFrame(val_answer_emb).to_csv('/content/drive/MyDrive/bertvalanswer.csv',index=False)
pd.DataFrame(test_clue_emb).to_csv('/content/drive/MyDrive/berttestclue.csv',index=False)
pd.DataFrame(test_answer_emb).to_csv('/content/drive/MyDrive/berttestanswer.csv',index=False)